In [2]:
# Import Dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Connecting to S3 bucket
import s3fs

# Create pandas dataframe from the csv file
df = pd.read_csv('https://jamesliu-databootcamp-bucket.s3.us-east-2.amazonaws.com/pseof_all.csv')

In [5]:
df.head()

,agg_level_pseo,inst_level,institution,degree_level,cip_level,cipcode,grad_cohort,grad_cohort_years,geo_level,geography,...,y10_grads_nme,status_y1_grads_emp,status_y1_grads_emp_instate,status_y5_grads_emp,status_y5_grads_emp_instate,status_y10_grads_emp,status_y10_grads_emp_instate,status_y1_grads_nme,status_y5_grads_nme,status_y10_grads_nme
0,38,I,00105100,5,A,0,0,3,N,0,...,7281.0,1,1,1,1,1,1,1,1,1
1,38,I,00105100,7,A,0,0,5,N,0,...,1149.0,1,1,1,1,1,1,1,1,1
2,38,I,00105100,17,A,0,0,5,N,0,...,191.0,1,1,1,1,1,1,1,1,1
3,38,I,00105100,18,A,0,0,5,N,0,...,256.0,1,1,1,1,1,1,1,1,1
4,38,I,00105200,2,A,0,0,5,N,0,...,NaN,1,1,1,1,-1,-1,1,1,-1


In [6]:
df.dtypes

agg_level_pseo                    int64
inst_level                       object
institution                      object
degree_level                      int64
cip_level                        object
cipcode                           int64
grad_cohort                       int64
grad_cohort_years                 int64
geo_level                        object
geography                         int64
ind_level                        object
industry                         object
y1_grads_emp                    float64
y1_grads_emp_instate            float64
y5_grads_emp                    float64
y5_grads_emp_instate            float64
y10_grads_emp                   float64
y10_grads_emp_instate           float64
y1_grads_nme                    float64
y5_grads_nme                    float64
y10_grads_nme                   float64
status_y1_grads_emp               int64
status_y1_grads_emp_instate       int64
status_y5_grads_emp               int64
status_y5_grads_emp_instate       int64


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['inst_level'] = le.fit_transform(df['inst_level'])
df.dtypes

agg_level_pseo                    int64
inst_level                        int64
institution                      object
degree_level                      int64
cip_level                        object
cipcode                           int64
grad_cohort                       int64
grad_cohort_years                 int64
geo_level                        object
geography                         int64
ind_level                        object
industry                         object
y1_grads_emp                    float64
y1_grads_emp_instate            float64
y5_grads_emp                    float64
y5_grads_emp_instate            float64
y10_grads_emp                   float64
y10_grads_emp_instate           float64
y1_grads_nme                    float64
y5_grads_nme                    float64
y10_grads_nme                   float64
status_y1_grads_emp               int64
status_y1_grads_emp_instate       int64
status_y5_grads_emp               int64
status_y5_grads_emp_instate       int64


In [9]:
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

In [10]:
# Object columns are dropped for analysis
drop_objects_df = df.drop(columns=['institution', 'cip_level','geo_level', 'industry','ind_level'])
drop_objects_df.head()

,agg_level_pseo,inst_level,degree_level,cipcode,grad_cohort,grad_cohort_years,geography,y1_grads_emp,y1_grads_emp_instate,y5_grads_emp,...,y10_grads_nme,status_y1_grads_emp,status_y1_grads_emp_instate,status_y5_grads_emp,status_y5_grads_emp_instate,status_y10_grads_emp,status_y10_grads_emp_instate,status_y1_grads_nme,status_y5_grads_nme,status_y10_grads_nme
0,38,0,5,0,0,3,0,49897.0,27741.0,32189.0,...,7281.0,1,1,1,1,1,1,1,1,1
1,38,0,7,0,0,5,0,14182.0,8311.0,8975.0,...,1149.0,1,1,1,1,1,1,1,1,1
2,38,0,17,0,0,5,0,2238.0,992.0,1401.0,...,191.0,1,1,1,1,1,1,1,1,1
3,38,0,18,0,0,5,0,1987.0,1208.0,1208.0,...,256.0,1,1,1,1,1,1,1,1,1
4,38,0,2,0,0,5,0,356.0,236.0,126.0,...,NaN,1,1,1,1,-1,-1,1,1,-1


In [12]:
scaled_data = data_scaler.fit_transform(drop_objects_df)